In [1]:
import pandas as pd
import numpy as np
import math

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

from IPython.display import display, HTML

init_notebook_mode(connected=True)

survey_v1 = pd.read_excel("Kassala_baseline_survey_for_CC_project_in_Sudan_final_results.xlsx", encoding = 'utf_8')
survey_v2 = pd.read_excel("Kassala_baseline_survey_for_CC_project_in_Sudan_final_results_v2.xlsx", encoding = 'utf_8')
reg_survey_complete = pd.concat([survey_v1, survey_v2])

# <center>Baseline survey results for the Locality of El Golid </center>
### <center>*Climate Change and Livelihoods situation of 15 villages located in El Golid Locality, Northern State*</center>

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

#bc_wages.head()
#reg_csv.info()
#bc_wages.describe()

In [3]:
# Functions

# Get ocurrences of each value for a multiple_select field
def get_ocurrences(df, column_name):
    values_sums = df.groupby(column_name).size()
    values_names = reduce(lambda x, y: x + y,map(lambda x: x.split(), values_sums.index.values))
    result = pd.Series()
    for i in values_names:
        result[i] = values_sums.loc[[i in idx for idx in values_sums.index]].sum()
    return result

def get_percentiles_values(sorted_serie, percentiles):
    if len(percentiles) == 0:
        return []
    else:
        quantity = sorted_serie.sum() * percentiles[0]
        for i in sorted_serie.index:
            if sorted_serie[i] >= quantity:
                return [i] + get_percentiles_values(sorted_serie, percentiles[1:])
            else:
                quantity -= sorted_serie[i]

def get_percentiles(sorted_serie, values):
    if len(values) == 0:
        return []
    else:
        result = 0
        for i in sorted_serie.index:
            if values[0] == i:
                return [float(result + sorted_serie[i]) / sorted_serie.sum()] + get_percentiles(sorted_serie, values[1:])
            elif values[0] < i:
                return [float(result) / sorted_serie.sum()] + get_percentiles(sorted_serie, values[1:])
            else:
                result += sorted_serie[i]
        return [1]

In [4]:
## File preprocessing
#Deleting 'demography:', 'house_assets:', 'incomes:', 'climate_change:' and those prefix which not provide useful info.

col_names = reg_survey_complete.columns
col_names = col_names.map(lambda x: x[11:] if x.startswith("demography:") else x)
col_names = col_names.map(lambda x: x[19:] if x.startswith("responding_section:") else x)
col_names = col_names.map(lambda x: x[20:] if x.startswith("responding_section2:") else x)
col_names = col_names.map(lambda x: x[13:] if x.startswith("house_assets:") else x)
col_names = col_names.map(lambda x: x[8:] if x.startswith("incomes:") else x)
col_names = col_names.map(lambda x: x[15:] if x.startswith("climate_change:") else x)
col_names = col_names.map(lambda x: x[11:] if x.startswith("intro_data:") else x)
col_names = col_names.map(lambda x: x[:11] + x[16:] if x.startswith("respondant_page:") else x)
col_names = col_names.map(lambda x: x[16:] if x.startswith("general_hh_data:") else x)
col_names = col_names.map(lambda x: x[6:] if x.startswith("males:") else x)
col_names = col_names.map(lambda x: x[8:] if x.startswith("females:") else x)
col_names = col_names.map(lambda x: x[11:] if x.startswith("importance:") else x)
col_names = col_names.map(lambda x: x[19:] if x.startswith("percentage_incomes:") else x)
col_names = col_names.map(lambda x: x[:10] + x[16:] if x.startswith("agri_gard:agri") else x)
col_names = col_names.map(lambda x: x[:10] + x[20:] if x.startswith("livestock:livestock_") else x)
col_names = col_names.map(lambda x: x[:10] + x[21:] if x.startswith("livestock:livestock") else x)
col_names = col_names.map(lambda x: x[:8] + x[17:] if x.startswith("fishing:fishing") else x)
col_names = col_names.map(lambda x: x[:7] + x[15:] if x.startswith("labour:labour") else x)
col_names = col_names.map(lambda x: x[:7] + x[14:] if x.startswith("labour:labour") else x)
col_names = col_names.map(lambda x: x[:12] + x[25:] if x.startswith("handicrafts:handicrafts") else x)
col_names = col_names.map(lambda x: x[:9] + x[19:] if x.startswith("forestry:forestry") else x)
col_names = col_names.map(lambda x: x[:6] + x[13:] if x.startswith("other:other") else x)
col_names = col_names.map(lambda x: x[7:] if x.startswith("coping:") else x)
col_names = col_names.map(lambda x: x[4:] if x.startswith(("cc1:", "cc2:", "cc3:", "ews:")) else x)
col_names = col_names.map(lambda x: x[10:] if x.startswith("renewable:") else x)
col_names = col_names.map(lambda x: x[12:] if x.startswith("agriculture_cc_effects:") else x)
col_names = col_names.map(lambda x: x[10:] if x.startswith("livestock_cc_effects:") else x)

In [5]:
# Replace ':' with '_' for compatibility reasons.
col_names = col_names.map(lambda x: x[:14] + '_' + x[15:] if x.startswith("geocoordinates:") else x)
col_names = col_names.map(lambda x: x[:9] + '_' + x[10:] if x.startswith("agri_gard:") else x)
col_names = col_names.map(lambda x: x[:9] + '_' + x[10:] if x.startswith("livestock:") else x)
col_names = col_names.map(lambda x: x[:7] + '_' + x[8:] if x.startswith("fishing:") else x)
col_names = col_names.map(lambda x: x[:6] + '_' + x[7:] if x.startswith("labour:") else x)
col_names = col_names.map(lambda x: x[:11] + '_' + x[12:] if x.startswith("handicrafts:") else x)
col_names = col_names.map(lambda x: x[:8] + '_' + x[9:] if x.startswith("forestry:") else x)
col_names = col_names.map(lambda x: x[:5] + '_' + x[6:] if x.startswith("other:") else x)
col_names = col_names.map(lambda x: x[:10] + '_' + x[11:] if x.startswith("cc_effects:") else x)
col_names = col_names.map(lambda x: x[:4] + '_' + x[5:] if x.startswith("meta:") else x)

col_names = col_names.map(lambda x: 'livestock_lost_goat' if x == 'livestock_lost_goats' else x)

print col_names.values

reg_survey_complete.columns = col_names

[u'deviceid' u'today' u'surveyor_name' u'number' u'village'
 u'geocoordinates_Latitude' u'geocoordinates_Longitude'
 u'geocoordinates_Altitude' u'geocoordinates_Accuracy' u'responding'
 u'respondant_gender' u'respondant_age' u'respondant_marital_status'
 u'respondant_head_hh' u'education' u'handicapped_member' u'number_wifes'
 u'number_wifes_aux' u'male_infant' u'male_child' u'male_child_school'
 u'male_teenager' u'male_teenager_school' u'male_adult' u'male_elderly'
 u'female_infant' u'female_child' u'female_child_school' u'female_teenager'
 u'female_teenager_school' u'female_adult' u'female_elderly'
 u'family_males' u'family_females' u'family_total' u'family_confirm'
 u'house_material' u'belongings' u'num_beds' u'kitchen_type' u'src_incomes'
 u'imp_agri' u'imp_garden' u'imp_livestock' u'imp_fishing' u'imp_labour'
 u'imp_forestry' u'imp_handcrafts' u'imp_other' u'perc_incomes_agri'
 u'perc_incomes_garden' u'perc_incomes_livestock' u'perc_incomes_fishing'
 u'perc_incomes_labour' u'perc_

In [6]:
# Processing NaN and void values

values = {
    'respondant_marital_status': 'no_answer',
    'house_material': 'no_answer',
    'kitchen_type': 'no_answer',
    'src_incomes': 'none',
    'use_ews': 'no_answer',
    'renewable_knowledge': 'no_answer',
    'solar_energy': 'no_answer',
    'livestock_selling': 'no_answer',
    'livestock_social': 'no_answer',
    'livestock_milk': 'no_answer',
    'livestock_meat': 'no_answer',
    'livestock_savings': 'no_answer',
    'labour_participants': 'no_answer',
    'agri_gard_water_harv_farmlands': 0,
    'agri_gard_land_quantity': 0,
    'cc_effects_sand_affected_farmlands': 0
}
reg_survey_complete.fillna(value=values, inplace=True)

In [7]:
# Creating villages dict
villages = reg_survey_complete['village'].unique()
villages_table = pd.read_excel("villages_northern.xlsx")
# Villages dictionary
villages_dict = dict()
ar_dict = villages_table.set_index('survey_code').to_dict()['Arabic name']
villages_dict['ar'] = ar_dict
villages_dict['en'] = villages_table.set_index('survey_code').to_dict()['English transcription']
#print repr(villages_dict).decode('unicode_escape')
reg_survey_complete['village'] = reg_survey_complete.village.map(villages_dict['en'])
print villages_dict

{'ar': {1: u'\u0644\u062a\u064a \u0642\u0633\u0645 1', 2: u'\u0644\u062a\u064a \u0642\u0633\u0645 2', 3: u'\u0644\u062a\u064a \u0642\u0633\u0645 4', 4: u'\u0627\u0644\u0645\u0642\u0627\u0648\u062f\u0629', 5: u'\u0643\u062f\u0643\u0648\u0644', 6: u'\u0646\u0627\u0648\u0627 \u0634\u0645\u0627\u0644', 7: u'\u0646\u0627\u0648\u0627 \u062c\u0646\u0648\u0628', 8: u'\u0628\u0633\u0644\u0627\u0646 \u0634\u0645\u0627\u0644', 9: u'\u0628\u0633\u0644\u0627\u0646 \u062c\u0646\u0648\u0628', 10: u'\u0627\u0644\u0632\u0631\u0627\u0626\u0628', 11: u'\u0645\u0644\u0648\u0627\u062f', 12: u'\u0631\u0648\u0645\u064a \u0628\u062d\u0631\u0649', 13: u'\u0631\u0648\u0645\u064a \u0642\u0628\u0644\u064a', 14: u'\u0631\u0648\u0645\u064a \u0648\u0633\u0637', 15: u'\u0623\u0645 \u0643\u0631\u0627\u0628\u064a\u062c', 16: u'\u0643\u0646\u0643\u0644\u0627\u0628'}, 'en': {1: u'Laty. D. 1', 2: u'Laty. D. 2', 3: u'Laty. D. 4', 4: u'Almagawda', 5: u'Kadacol', 6: u'North Nawa', 7: u'South Nawa', 8: u'North Baslan', 9: u'S

In [8]:
# For answered surveys
reg_survey = reg_survey_complete.loc[reg_survey_complete['responding'].str.match('yes')].copy()

In [9]:
# Creating num of LH strategies
reg_survey['num_lh_strats'] = reg_survey['src_incomes'].map(lambda x: len(x.split()))

In [10]:
# Separate male and female
reg_survey_male = reg_survey.loc[reg_survey['respondant_gender'].str.match('male')].copy()
reg_survey_female = reg_survey.loc[reg_survey['respondant_gender'].str.match('female')].copy()

In [11]:
## Separate per HoHH gender

# Respondent is HH
reg_survey['married'] = reg_survey['respondant_marital_status'].str.match('married')
# Females HoHH
reg_survey_female_head = reg_survey[reg_survey['respondant_gender'].str.match('female') &
                                      reg_survey['respondant_head_hh'].str.match('yes')].copy()

# Females noHoHH
reg_survey_female_no_head = reg_survey[reg_survey['respondant_gender'].str.match('female') &
                                      reg_survey['respondant_head_hh'].str.match('no')].copy()

# Males HoHH + Female noHoHH married
reg_survey_male_head = reg_survey[reg_survey['respondant_gender'].str.match('male') &
                                      reg_survey['respondant_head_hh'].str.match('yes')].copy()
reg_survey_male_head = pd.concat([reg_survey_male_head,
                                  reg_survey_female_no_head[reg_survey_female_no_head['married']]])


# Add females nHoHH nM with male_adult to male_head
reg_survey_female_nHoHH_nM = reg_survey_female_no_head[~reg_survey_female_no_head['married']]
reg_survey_male_head = pd.concat([reg_survey_male_head,
                                  reg_survey_female_nHoHH_nM[reg_survey_female_nHoHH_nM['male_adult'] > 0]])

# Add females nHoHH nM with male_adult to female_head
reg_survey_female_head = pd.concat([reg_survey_female_head,
                                    reg_survey_female_nHoHH_nM[reg_survey_female_nHoHH_nM['male_adult'] == 0]])

# Add males nHoHH with male_adult to male_head
reg_survey_male_nHoHH = reg_survey[reg_survey['respondant_gender'].str.match('male') &
                                      reg_survey['respondant_head_hh'].str.match('no')].copy()
reg_survey_maleHoHH = pd.concat([reg_survey_male_head,
                                  reg_survey_male_nHoHH[reg_survey_male_nHoHH['male_adult'] > 0]])

# Add females nHoHH with male_adult to female_head
reg_survey_femHoHH = pd.concat([reg_survey_female_head,
                                  reg_survey_male_nHoHH[reg_survey_male_nHoHH['male_adult'] == 0]])

In [12]:
def create_graphs_HoHH(data_func, values_func, labels_func, sum_func, title):
    create_three_pie_graphs((reg_survey_femHoHH, reg_survey, reg_survey_maleHoHH), data_func, values_func,
                            labels_func, sum_func, title,
                            "Female Head of Household", .04,
                            "Global", .5,
                            "Male Head of Household", .94)

def create_graphs_respondent(data_func, values_func, labels_func, sum_func, title):
    create_three_pie_graphs((reg_survey_female, reg_survey, reg_survey_male), data_func, values_func,
                            labels_func, sum_func, title,
                            "Female Respondent", .07,
                            "Global", .5,
                            "Male Respondent", .91)
    
def create_three_pie_graphs((df1, df2, df3), data_func, values_func,
                            labels_func, sum_func, title,
                            title1, title1_x_offset,
                            title2, title2_x_offset,
                            title3, title3_x_offset):
    data_fem = data_func(df1)
    data_global = data_func(df2)
    data_male = data_func(df3)

    fig = {
        "layout": {
            "title": title,
            "annotations": [
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title1 + " (" + str(sum_func(data_fem)) + ")",
                    "text": title1,
                    "x": title1_x_offset,
                    "y": 1
                },
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title2 + " (" + str(sum_func(data_global)) + ")",
                    "text": title2,
                    "x": title2_x_offset,
                    "y": 1
                },
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title3 + " (" + str(sum_func(data_male)) + ")",
                    "text": title3,
                    "x": title3_x_offset,
                    "y": 1
                }
            ]
        },
        "data": [
            {
                "values": values_func(data_fem),
                "labels": labels_func(data_fem),
                "domain": {
                    "x": [0, .32]
                },
                "type": "pie",
                "sort": False
            },
            {
                "values": values_func(data_global),
                "labels": labels_func(data_global),
                "domain": {
                    "x": [.34, .64]
                },
                "type": "pie",
                "sort": False
            },
            {
                "values": values_func(data_male),
                "labels": labels_func(data_male),
                "domain": {
                    "x": [.66, .98]
                },
                "type": "pie",
                "sort": False
            }
        ]
    }

    iplot(fig)

def create_stacked_bars_graph((df1, df2), data_func, x_func, y_func, title,
                              title_df1, title_df2):
    data_global = data_func(pd.concat([df1, df2]))
    data_fem = data_func(df1)
    data_male = data_func(df2)

    fig = {
        "layout": {
            "title": title,
            "xaxis": {
                "tickangle": -45
            },
            "barmode": "stack",
            "width": 700,
            "annotations": []
        },
        "data": [
            {
                "x": x_func(data_fem),
                "y": y_func(data_fem),
                "type": "bar",
                "name": title_df1,
                "marker": {
                    "color": 'rgb(225,225,258)',
                    "line": {
                        "color": 'rgb(0,0,192)',
                        "width": 1.5
                    }
                },
            },
            {
                "x": x_func(data_male),
                "y": y_func(data_male),
                "type": "bar",
                "name": title_df2,
                "marker": {
                    "color": 'rgb(258,225,225)',
                    "line": {
                        "color": 'rgb(192,0,0)',
                        "width": 1.5
                    }
                },
            }
        
        ]
    }


#    for i in range(0, len(kitchen_type)):
#        fig["layout"]["annotations"].append(
#            {
#                 "x": map(lambda x: x.capitalize(), kitchen_perc.index.values)[i],
#                 "y": kitchen_perc.values[i] + 1.5,
#                "x": pd.concat([df1, df2])[i],
#                "y": y_func(data_global)[i] + 1.5,
#                "text": str(data_global.values[i]) + "%",
#                "font": dict(
#                    family='Arial',
#                    size=14,
#                    color='rgba(0, 0, 100, 1)'
#                ),
#                "showarrow": False,
#    
#            }
#        )
#
#
    iplot(fig)

## General data

Number of Surveys:

In [13]:
print "Total:" + str(reg_survey_complete['deviceid'].count())
print ""
total_surveys = reg_survey['deviceid'].count()
print "Responding:" + str(total_surveys)
print ""

Total:399

Responding:384



### Respondents per gender and village

In [14]:
surveys_village = reg_survey.groupby('village').size()
surveys_village_male = reg_survey_male.groupby('village').size()
surveys_village_female = reg_survey_female.groupby('village').size()
respondants_gender = pd.DataFrame(surveys_village, columns= ['Total'])
respondants_gender['Male'] = surveys_village_male
respondants_gender['Male %'] = (surveys_village_male*100 / surveys_village).round(2)
respondants_gender['Female'] = surveys_village_female
respondants_gender['Female %'] = (surveys_village_female*100 / surveys_village).round(2)
respondants_gender = respondants_gender.reindex(['Male', 'Male %', 'Female', 'Female %', 'Total'], axis=1)
respondants_gender.loc['Total', 'Total'] = respondants_gender['Total'].sum()
respondants_gender.loc['Total', 'Male'] = respondants_gender['Male'].sum()
respondants_gender.loc['Total', 'Male %'] = round(100*respondants_gender.loc['Total', 'Male'] / respondants_gender.loc['Total', 'Total'], 2)
respondants_gender.loc['Total', 'Female'] = respondants_gender['Female'].sum()
respondants_gender.loc['Total', 'Female %'] = round(100*respondants_gender.loc['Total', 'Female'] / respondants_gender.loc['Total', 'Total'], 2)
respondants_gender.fillna(value={'Male': 0}, inplace=True)
respondants_gender['Male'] = respondants_gender['Male'].astype(int)
respondants_gender['Female'] = respondants_gender['Female'].astype(int)
respondants_gender['Total'] = respondants_gender['Total'].astype(int)

display(respondants_gender)

,Male,Male %,Female,Female %,Total
village,,,,,
Almagawda,4,16.67,20,83.33,24
Alzaraib,0,NaN,1,100.00,1
Kadacol,0,NaN,35,100.00,35
Laty. D. 1,7,12.07,51,87.93,58
Laty. D. 2,0,NaN,22,100.00,22
Laty. D. 4,3,8.33,33,91.67,36
Medium Romy,5,17.24,24,82.76,29
Molwad,2,7.41,25,92.59,27
North Baslan,0,NaN,22,100.00,22


In [15]:
surveys_village_maleHoHH = reg_survey_maleHoHH.groupby('village').size()
surveys_village_femHoHH = reg_survey_femHoHH.groupby('village').size()
HoHH_gender = pd.DataFrame(surveys_village, columns= ['Total'])
HoHH_gender['Male'] = surveys_village_maleHoHH
HoHH_gender['Male %'] = (surveys_village_maleHoHH*100 / surveys_village).round(2)
HoHH_gender['Female'] = surveys_village_femHoHH
HoHH_gender['Female %'] = (surveys_village_femHoHH*100 / surveys_village).round(2)
HoHH_gender = HoHH_gender.reindex(['Male', 'Male %', 'Female', 'Female %', 'Total'], axis=1)
HoHH_gender.loc['Total', 'Total'] = HoHH_gender['Total'].sum()
HoHH_gender.loc['Total', 'Male'] = HoHH_gender['Male'].sum()
HoHH_gender.loc['Total', 'Male %'] = round(100*HoHH_gender.loc['Total', 'Male'] / HoHH_gender.loc['Total', 'Total'], 2)
HoHH_gender.loc['Total', 'Female'] = HoHH_gender['Female'].sum()
HoHH_gender.loc['Total', 'Female %'] = round(100*HoHH_gender.loc['Total', 'Female'] / HoHH_gender.loc['Total', 'Total'], 2)
HoHH_gender.fillna(value={'Male': 0}, inplace=True)
HoHH_gender['Male'] = HoHH_gender['Male'].astype(int)
HoHH_gender['Female'] = HoHH_gender['Female'].astype(int)
HoHH_gender['Total'] = HoHH_gender['Total'].astype(int)

display(HoHH_gender)

,Male,Male %,Female,Female %,Total
village,,,,,
Almagawda,8,33.33,16,66.67,24
Alzaraib,0,NaN,1,100.00,1
Kadacol,9,25.71,26,74.29,35
Laty. D. 1,19,32.76,39,67.24,58
Laty. D. 2,7,31.82,15,68.18,22
Laty. D. 4,10,27.78,26,72.22,36
Medium Romy,9,31.03,20,68.97,29
Molwad,10,37.04,17,62.96,27
North Baslan,8,36.36,14,63.64,22


### HHs Composition

In [16]:
print reg_survey_femHoHH['deviceid'].count()
print reg_survey_maleHoHH['deviceid'].count()

273
111


In [17]:
create_graphs_HoHH(lambda x: x.groupby('education').size(),
                 lambda x: x.values,
                 lambda x: x.index.values,
                   lambda x: x.sum(),
                 "Education level of the head of household")


In [18]:
#hey = reg_survey.groupby(['village', 'house_material']).size()

#house_materials_df = pd.DataFrame(index=house_materials_index, columns=villages, data=[house_materials_general])
#print house_materials_df

create_graphs_HoHH(lambda x: pd.crosstab(x.village, x.house_material),
                 lambda x: x.sum().values,
                 lambda x: x.columns.values,
                   lambda x: x.sum().sum(),
                 "House materials")

#fig_house_materials2 = {
#    "layout": {
#        "title": {
#            "font": {
#                    "size": 20
#            },
#            "text": "House materials"
#        }
#    },
#    "data": []
#}
#
#house_materials_df = pd.crosstab(reg_survey.village, reg_survey.house_material)
##print range(len(villages_dict['en'].keys())-1)
##print house_materials_df
#for i in range(len(villages_dict['en'].keys())):
#    x = .2*(i%5)
#    y = 0.35*(i/5)
#    fig_house_materials2['data'].append(
#        {
#
#            "values": house_materials_df.iloc[i-1].values,
#            "labels": house_materials_df.columns.values,
#             "domain": {
#                "x": [x, x+.18],
#                 "y": [y, y+.3]
#            },
#            "type": "pie",
#            "sort": False
#        })
#
#iplot(fig_house_materials2)

###  Mean Household size

In [19]:
mean_hh_size = reg_survey.family_total.mean()
print str(mean_hh_size) + " persons per household"

5.7890625 persons per household


## Climate change awareness

In [20]:
create_graphs_respondent(lambda x: x.groupby('cc_knowledge').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring CC knowledge")

In [21]:
create_graphs_respondent(lambda x: x.groupby('taking_action_cc').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring taking actions against CC")

#### EWS awareness

In [22]:
create_graphs_respondent(lambda x: x.groupby('ews_knowledge').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring EWS knowledge")

In [23]:
create_graphs_respondent(lambda x: x.groupby('use_ews').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households using EWS information")

#### Renewable energy awareness

In [24]:
create_graphs_respondent(lambda x: x.groupby('renewable_knowledge').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring renewable energies knowledge")

In [25]:
create_graphs_respondent(lambda x: x.groupby('solar_energy').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households using solar energies")

#### Cooking facilities

In [26]:
kitchen_type = reg_survey.groupby('kitchen_type').size()
kitchen_type_male = get_ocurrences(reg_survey_maleHoHH, 'kitchen_type').sort_values(ascending = False)
kitchen_type_female = get_ocurrences(reg_survey_femHoHH, 'kitchen_type').sort_values(ascending = False)
kitchen = pd.crosstab(reg_survey.village, reg_survey.kitchen_type)
#display(kitchen)

kitchen_perc_male = 100*kitchen_type_male/total_surveys
kitchen_perc_female = 100*kitchen_type_female/total_surveys
kitchen_perc = 100*kitchen_type/total_surveys
kitchen_perc = kitchen_perc.round(2)
#print kitchen_perc

create_stacked_bars_graph((reg_survey_femHoHH, reg_survey_maleHoHH),
                              lambda x: 100 * get_ocurrences(x, 'kitchen_type').sort_values(ascending = False) / total_surveys,
                              lambda x: map(lambda y: y.capitalize(), x.index.values),
                              lambda x: x.values,
                              "Cooking techniques",
                              "Female Head of Household",
                              "Male Head of Household")

## Livelihoods

Number of households practicing each livelihood

In [27]:
#reg_survey.loc[reg_survey['src_incomes'].str.contains("agriculture"), 'agriculture'] = 'yes'
#reg_survey.loc[reg_survey['src_incomes'].str.contains("livestock"), 'livestock'] = 'yes'
#reg_survey.loc[reg_survey['src_incomes'].str.contains("labour"), 'labour'] = 'yes'
#importance = reg_survey.groupby(['agriculture', 'livestock', 'labour'])['perc_incomes_agri'].agg('mean')

#agric = reg_survey[reg_survey['src_incomes'].str.contains("agriculture")]['deviceid'].count()

src_incomes_sums = reg_survey.groupby('src_incomes').size()
lh_values = get_ocurrences(reg_survey, 'src_incomes')
lh_types = pd.DataFrame({"Households": lh_values})
#lh_types["Percentage"] = lh_types.Households.map(lambda x: str(round(100*x/total_surveys))) + " %"

print ""
print lh_values.sort_values(ascending = False)


livestock      234
forestry       130
other           43
handicrafts     30
labour          19
agriculture     19
dtype: int64


In [28]:
lh_perc = 100*lh_values/total_surveys
lh_perc = lh_perc.round(2)

create_stacked_bars_graph((reg_survey_femHoHH, reg_survey_maleHoHH),
                              lambda x: 100 * get_ocurrences(x, 'src_incomes') / total_surveys,
                              lambda x: map(lambda y: y.capitalize(), x.index.values),
                              lambda x: x.values,
                              "Livelihoods Strategies",
                              "Female Head of Household",
                              "Male Head of Household")

create_stacked_bars_graph((reg_survey_female, reg_survey_male),
                              lambda x: 100 * get_ocurrences(x, 'src_incomes') / total_surveys,
                              lambda x: map(lambda y: y.capitalize(), x.index.values),
                              lambda x: x.values,
                              "Livelihoods Strategies",
                              "Female respondent",
                              "Male respondent")

livelihoods_strategies = {
    "layout": {
        "title": "Livelihoods Strategies",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), lh_perc.index.values),
            "y": lh_perc.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            },
        }
    ]
}


for i in range(0, len(lh_values)):
    livelihoods_strategies["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), lh_perc.index.values)[i],
            "y": lh_perc.values[i] + 1.5,
            "text": str(lh_perc.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(livelihoods_strategies)

Main livelihoods per household

In [29]:
print src_incomes_sums.sort_values(ascending = False)
reg_survey['num_lh_strats'] = reg_survey['src_incomes'].map(lambda x: len(x.split()))
create_graphs_HoHH(lambda x: x.groupby('num_lh_strats').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Number of livelihoods Strategies")

src_incomes
livestock                         162
forestry                           90
livestock forestry                 25
other                              24
livestock handicrafts              18
labour                             16
livestock other                    11
agriculture                         9
agriculture livestock               6
livestock forestry handicrafts      5
forestry handicrafts                3
livestock forestry other            2
handicrafts                         2
forestry other                      2
livestock labour                    2
labour forestry                     1
livestock handicrafts other         1
agriculture other                   1
agriculture livestock other         1
agriculture livestock forestry      1
agriculture forestry                1
handicrafts other                   1
dtype: int64


- 277 out of 399 hh, i.e. 69.42%, lives only on livestock, forestry or both.

### Livestock

Number of interviewed households owning each type of livestock

In [30]:
livestock_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("livestock")]

participants = livestock_set.groupby('livestock_participants').size()
participants_goats = livestock_set.loc[livestock_set['livestock_type'].str.contains("goat")].groupby('livestock_participants').size()
#print participants_goats

lvstk_types = ['camel', 'cattle', 'goat', 'sheep', 'chicken']

types_lvstck = get_ocurrences(livestock_set, 'livestock_type')
livestock_women = livestock_set.loc[livestock_set['livestock_participants'] == "women"]
type_women =  livestock_women.groupby('livestock_type').size()
#livestock_set = livestock_set.copy()
for i in lvstk_types:
    livestock_set[i + '_reduced'] = livestock_set['livestock_lost_' + i] / (livestock_set['livestock_number_' + i] + livestock_set['livestock_number_' + i])
#print ("Types for women")
#print(type_women)
#print livestock_set.groupby('camel_reduced').size()
goats = livestock_set['livestock_number_goat']

sheeps = livestock_set['livestock_number_sheep'].dropna()
camels = livestock_set['livestock_number_camel'].dropna()
chickens = livestock_set['livestock_number_chicken'].dropna()

livestock_set.groupby('livestock_training').size()



livestock_training
no     103
yes    131
dtype: int64

In [31]:
perc_lvstck = 100*types_lvstck/livestock_set['deviceid'].count()
perc_lvstck = perc_lvstck.round(2)
fig_types_lvstck = {
    "layout": {
        "title": "Livestock types",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), perc_lvstck.index.values),
            "y": perc_lvstck.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            }
        }
    ]
}


for i in range(0, len(types_lvstck)):
    fig_types_lvstck["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), perc_lvstck.index.values)[i],
            "y": perc_lvstck.values[i] + 1.5,
            "text": str(perc_lvstck.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(fig_types_lvstck)

In [32]:
prueba = pd.crosstab(livestock_set.livestock_number_goat, livestock_set.livestock_number_sheep).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba = prueba[prueba.number != 0]
prueba = prueba[prueba.livestock_number_goat <= 100]
prueba = prueba[prueba.livestock_number_sheep <= 100]
#display(prueba)
goats_vs_sheeps = {
    "layout": {
        "title": "Relation between the number of goats and sheeps"
    },
    "data": [
        {

            "x": prueba.livestock_number_goat,
            "y": prueba.livestock_number_sheep,
            "mode": "markers",
            "marker": {
                "size": prueba.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(goats_vs_sheeps)

With a few exceptions, the number of goats per household is proportional to the number of sheeps. So to define wealth groups for livestock tenants the number of goats will be the main variable.

In [36]:
quantiles = [.10, .35, .97, 1]
q = goats.quantile(quantiles)
print "Quantiles:"
print q
goats1 = goats[goats < q[quantiles[0]]].dropna()
goats2 = goats[goats.between(q[quantiles[0]], q[quantiles[1]])].dropna()
goats3 = goats[goats.between(q[quantiles[1]], q[quantiles[2]])].dropna()
fig = ff.create_distplot([goats1.values, goats2.values, goats3.values], ['poor', 'medium', 'rich'])
iplot (fig)

Quantiles:
0.10      5.0
0.35      9.0
0.97     35.0
1.00    110.0
Name: livestock_number_goat, dtype: float64


#### Quantiles for goats
- Poors: 35% of households have less than 10 goats and less than 10 sheeps
- Medium: 62% of households have between 10 and 35 goats and between 20 and 30 sheeps
- Richs: 3% of households have more than 35 goats and/or more than 30 sheeps

In [34]:
livestock_uses = ["selling", "social", "milk", "meat", "savings"]
goats_uses = ["selling", "social", "milk", "meat", "savings"]
sheeps_uses = ["selling", "social", "milk", "meat", "savings"]
livestock_uses_df = livestock_set[map(lambda x: "livestock_" + x, livestock_uses)]
livestock_uses_df.fillna('no')
print "USES TOTAL LIVESTOCK"
print livestock_uses_df.apply(pd.Series.value_counts)
goats_uses = livestock_set.loc[livestock_set['livestock_type'].str.contains("goat")][map(lambda x: "livestock_" + x, goats_uses)]
print ""
print "USES TOTAL GOATS"
print goats_uses.apply(pd.Series.value_counts)
sheeps_uses = livestock_set.loc[livestock_set['livestock_type'].str.contains("sheep")][map(lambda x: "livestock_" + x, sheeps_uses)]
print ""
print "USES TOTAL SHEEPS"
print sheeps_uses.apply(pd.Series.value_counts)

USES TOTAL LIVESTOCK
           livestock_selling  livestock_social  livestock_milk  \
no                      79.0                76               4   
no_answer                NaN                 2               3   
yes                    155.0               156             227   

           livestock_meat  livestock_savings  
no                     61                 32  
no_answer               3                  7  
yes                   170                195  

USES TOTAL GOATS
           livestock_selling  livestock_social  livestock_milk  \
no                      74.0                72               3   
no_answer                NaN                 2               3   
yes                    144.0               144             212   

           livestock_meat  livestock_savings  
no                     54                 30  
no_answer               3                  7  
yes                   161                181  

USES TOTAL SHEEPS
           livestock_selling  livest

### Laborers

In [35]:
labour_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("labour")]
print get_ocurrences(labour_set, 'labour_sector')
labour_set.loc[labour_set['labour_participants'].str.contains('women')][['labour_participants', 'respondant_gender', 'respondant_age', 'respondant_head_hh', 'male_teenager', 'male_adult', 'male_elderly', 'labour_sector']]
labour_incomes = labour_set.groupby('labour_daily_salary').size()
mean_salary = labour_set['labour_daily_salary'].mean()
labour_incomes_others = labour_set.loc[labour_set['labour_sector'] == 'other'].groupby('labour_daily_salary').size()
sectors_100 = labour_set.loc[labour_set['labour_daily_salary'] == 100].groupby('labour_sector').size()
print labour_set['deviceid'].count()
print mean_salary
print labour_incomes_others
create_graphs_HoHH(lambda x: x.loc[reg_survey['src_incomes'].str.contains("labour")].groupby('labour_work_term').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Temporality of laborers")
create_graphs_HoHH(lambda x: x.loc[reg_survey['src_incomes'].str.contains("labour")].groupby('labour_participants').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Gender of laborers")

other    17
dtype: int64
19
30.8333333333
labour_daily_salary
0.0     4
30.0    1
40.0    3
50.0    3
70.0    1
dtype: int64


ValueError: cannot reindex from a duplicate axis

### Agriculture

In [ ]:
agri_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("agriculture")].copy()

land_quantity = agri_set.groupby('agri_gard_land_quantity').size()
use_quantity = agri_set.groupby('agri_gard_use_quantity').size()

d = {'land': land_quantity_group, 'use': use}
use_land = pd.DataFrame(d)
hey = use_quantity.index.map(lambda x: str(x))
use_quantity.index = hey
print use_quantity.index
display(use_quantity)
display(use_land)
display(land_quantity)

In [ ]:
values = [1,5, land_quantity.index[-1]]
per2 = get_percentiles(land_quantity, values)
percentiles = pd.Series(values, index = per2)

print percentiles

def plot_wealth_criteria(groupby_data, values):
    poor = groupby_data[groupby_data.index <= values[0]]
    medium = groupby_data[groupby_data.filter(groupby_data.all(groupby_data > values[0], groupby_data <= values[1]))]
    rich = groupby_data[groupby_data.index.between(values[1],values[2])]
    print medium

plot_wealth_criteria(land_quantity, values)

fig_distribution = {
    "layout": {
        "title": "Title",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": poor.index.values,
            "y": poor.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            }
        }
    ]
}

iplot(fig_distribution)



In [ ]:
quantiles = [.335, .915, 1]
q = goats.quantile(quantiles)
print "Quantiles:"
print q
goats1 = goats[goats < q[quantiles[0]]].dropna()
goats2 = goats[goats.between(q[quantiles[0]], q[quantiles[1]])].dropna()
goats3 = goats[goats.between(q[quantiles[1]], q[quantiles[2]])].dropna()
fig = ff.create_distplot([goats1.values, goats2.values, goats3.values], ['poor', 'medium', 'rich'])
iplot (fig)

In [ ]:
use_vs_have = pd.crosstab(agri_set.agri_gard_land_quantity, agri_set.agri_gard_use_quantity).stack().reset_index(name='feddans')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
use_vs_have = use_vs_have[use_vs_have.feddans != 0]
use_vs_have = use_vs_have[use_vs_have.agri_gard_use_quantity <= 10]
use_vs_have = use_vs_have[use_vs_have.agri_gard_land_quantity <= 10]
#prueba = prueba[prueba.livestock_number_goat <= 100]
#prueba = prueba[prueba.livestock_number_sheep <= 100]
#display(prueba)
use_vs_have = {
    "layout": {
        "title": "Relation between the quantity of land and how much is used"
    },
    "data": [
        {

            "x": use_vs_have.agri_gard_land_quantity,
            "y": use_vs_have.agri_gard_use_quantity,
            "mode": "markers",
            "marker": {
                "size": use_vs_have.feddans
            },
            "type": "scatter"
        },
        {

            "x": [0,10],
            "y": [0,10],
            "mode": "lines"
        }
    ]
}
iplot(use_vs_have)

In [ ]:
# Inconsistencies
# print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_water_harv_farmlands']
# print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_land_quantity']

agri_set['perc_rainfed_farmlands'] = agri_set['agri_gard_water_harv_farmlands'] / agri_set['agri_gard_land_quantity']

# Estimation to correct the inconsistencies: All feddan have water harvesting access
feddans_med_poor = agri_set.loc[agri_set.agri_gard_use_quantity <= 5]['agri_gard_use_quantity'].sum()

print feddans_med_poor / agri_set.loc[agri_set.agri_gard_use_quantity <= 5, 'deviceid'].count()
print feddans_med_poor / total_surveys

#print agri_set.groupby('perc_rainfed_farmlands').size()
#print agri_set.groupby('agri_gard_farm_tech').size()
#print agri_set.groupby('agri_gard_water_harv_farmlands').size()
all_farm_rainfed = agri_set.loc[agri_set['perc_rainfed_farmlands'] == 1].groupby('agri_gard_use_quantity').size()
agri_set['agri_gard_water_harv_farmlands'].sum()
print agri_set.loc[agri_set.agri_gard_use_quantity > agri_set.agri_gard_land_quantity]['deviceid'].count()
print agri_set.loc[agri_set.agri_gard_land_property == 'own'].groupby('agri_gard_use_quantity').size()
print agri_set.loc[agri_set.agri_gard_land_property == 'rent'].groupby('agri_gard_use_quantity').size()
print agri_set.groupby('agri_gard_land_quantity').size()
print agri_set.groupby('cc_effects_sand_affected_farmlands').size()

agri_set.groupby('agri_gard_training').size()

In [ ]:
agri_set_med_poor = agri_set.loc[agri_set.agri_gard_use_quantity <= 5].copy()
agri_set_med_poor.loc[agri_set_med_poor['agri_gard_protected_farmlands'] > agri_set_med_poor['agri_gard_use_quantity'], 'agri_gard_protected_farmlands'] = agri_set_med_poor['agri_gard_use_quantity']
agri_set_med_poor['perc_land_protected'] = agri_set_med_poor.agri_gard_protected_farmlands / agri_set_med_poor.agri_gard_land_quantity
print agri_set_med_poor.groupby('agri_gard_use_quantity').size()
print agri_set_med_poor['agri_gard_protected_farmlands'].sum() / agri_set_med_poor['agri_gard_use_quantity'].sum()
print agri_set_med_poor.groupby('perc_land_protected').size()
print agri_set_med_poor.groupby('perc_land_protected').size().sum()

In [ ]:
#crop_types = ['sorghum', 'wheat', 'bean', 'palm', 'fodder', 'watermelon', 'potato', 'tomato', 'onion', 'cucumber', 'okra', 'other']

types_agri = get_ocurrences(agri_set, 'agri_gard_agri_crops')
print types_agri

In [ ]:
farm_tools = reg_survey.groupby('agri_gard_farm_tools').size()
farm_tech = reg_survey.groupby('agri_gard_farm_tech').size()
seeds = reg_survey.groupby('agri_gard_seeds').size()

create_graphs_HoHH(lambda x: x.groupby('agri_gard_farm_tools').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by Head of Household")

create_graphs_respondent(lambda x: x.groupby('agri_gard_farm_tools').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by respondent")

In [ ]:
create_graphs_HoHH(lambda x: x.groupby('agri_gard_farm_tech').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by Head of Household")

create_graphs_respondent(lambda x: x.groupby('agri_gard_farm_tech').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by respondent")

In [ ]:
create_graphs_HoHH(lambda x: x.groupby('agri_gard_seeds').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by Head of Household")

create_graphs_respondent(lambda x: x.groupby('agri_gard_seeds').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by respondent")

In [ ]:
crops = get_ocurrences(agri_set, 'agri_gard_agri_crops')
print crops

### Forestry (charcoal production)

In [ ]:
forestry_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("forestry")]
print forestry_set.groupby('forestry_forestry_type').size()
print get_ocurrences(forestry_set, 'forestry_forestry_type')
forestry_participants =  forestry_set.groupby('forestry_participants').size()
#forestry_set.loc[forestry_set['forestry_participants'].str.contains('women')][['forestry_participants', 'respondant_gender', 'respondant_age', 'respondant_head_hh', 'male_teenager', 'male_adult', 'male_elderly', 'labour_sector']]
forestry_land_property = forestry_set.groupby('forestry_land_property').size()

print forestry_participants
print forestry_land_property

### Other

In [ ]:
other_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("other")]
other_set.groupby('other_training').size()

## Climate change effects

In [ ]:
cc_effects_on_rain = get_ocurrences(reg_survey, 'changed_rainy_season')

print cc_effects_on_rain

In [ ]:
sand_mov_sums = reg_survey.groupby('sand_movements').size()
wind_changes = get_ocurrences(reg_survey, 'sand_movements')

print wind_changes
print sand_mov_sums

create_graphs_respondent(lambda x: x.groupby('sand_movements').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Respondents asked about wind changes in las ten years")

In [ ]:
agri_set_med_poor.loc[agri_set_med_poor['cc_effects_sand_affected_farmlands'] > agri_set_med_poor['agri_gard_use_quantity'], 'cc_effects_sand_affected_farmlands'] = agri_set_med_poor['agri_gard_use_quantity']
agri_set_med_poor['perc_land_affected'] = agri_set_med_poor.cc_effects_sand_affected_farmlands / agri_set_med_poor.agri_gard_land_quantity
print agri_set_med_poor['cc_effects_sand_affected_farmlands'].sum() / agri_set_med_poor['agri_gard_use_quantity'].sum()
print agri_set_med_poor.groupby('perc_land_affected').size()
print agri_set_med_poor.groupby('perc_land_affected').size().sum()

In [ ]:
temps_changes = get_ocurrences(reg_survey, 'temps_change')

print temps_changes

In [ ]:
effects_on_lvstck = get_ocurrences(reg_survey, 'cc_effects_grassland_availability').sort_values(ascending = False)
lvstck_perc = 100*effects_on_lvstck/livestock_set['deviceid'].size
lvstck_perc = lvstck_perc.round(2)
fig_effects_on_lvstck = {
    "layout": {
        "title": "Trend of the availability of grassland in the last 5 years",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), lvstck_perc.index.values),
            "y": lvstck_perc.values,
            "type": "bar",
            "marker": {
                "color": 'rgb(25,158,25,0.6)',
                "line": {
                    "color": 'rgb(0,70,0)',
                    "width": 1.5
                }
            },
        }
    ]
}


for i in range(0, len(effects_on_lvstck)):
    fig_effects_on_lvstck["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), lvstck_perc.index.values)[i],
            "y": lvstck_perc.values[i] + 1.7,
            "text": str(lvstck_perc.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(0, 100, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(fig_effects_on_lvstck)
